# Partie Thermique

---

In [41]:
from numpy import linspace, zeros
from numpy import linalg
from numpy import ndarray
import plotly.express as px

In [42]:
#Initialisation des constante

lbda = 0.96
rho = 2550
Cp = 840

N = 100
R = 1

t = 10
Nt = 10000

Text = linspace(1770,1670, Nt)
Tref = 1770

In [43]:
#Calcule des constantes
W = lbda/(rho * Cp)
dr = R / N
dt = t / Nt

T = []

#Initialisation de la temperaure
for i in range(Nt):
    T.append( ([Tref]*N)  + [Text[i]] )



def sq(x):
    return x*x

In [44]:

for m in range(1, Nt):

    T[m][0] = W * dt * (T[m-1][1] - T[m-1][0])/sq(dr) + T[m-1][0]

    for i in range(1,N):
        T[m][i] = W * dt * ( (T[m-1][i+1] - 2*T[m-1][i] + T[m-1][i-1])/sq(dr) + (4*(T[m-1][i+1] - T[m-1][i])) / ( sq(dr) * (2*i - 1))) + T[m-1][i]

In [45]:
#Tim = [ [ (T[m][i] + T[m][i-1])/2 for i in range(1, N+0)] for m in range(0, Nt)]
totalDT = [ [Tref - T[m][i] for i in range(len(T[0])-1)] for m in range(len(T))]

In [46]:
X = linspace(0, R, N)
Y = T[-1][:-1]

px.line(x = X , y = Y)

In [47]:
totalDT = []
X = linspace(0, R, N)
totalDT.append([ -(r*r/(R*R) * 100) for r in X] )
#totalDT.append([0 for i in X])

#px.line(x = X, y = DT, title=r"Temperature en focntion de R")

# Partie Mécanique

---

In [48]:
p = 1

E = 71.5e9
nu = 0.21
kc = E/(3*(1-2*nu))
muc= E/(2*(1+nu))
alpha_c=9e-6

In [49]:
def k(i) -> float:
    return kc

def mu(i) -> float:
    return muc


def A(i) -> float:
    A = 4/(9*k(i)) + 1/(12 * mu(i))
    return A

def B(i) -> float:
    B = 1/(3*k(i))
    return B

def alpha(i) -> float:
    return alpha_c


## Construction de M et de R

In [50]:

temps = 0#Nt-1
DT = totalDT[temps]

M = [[0 for i in range(2*N)] for j in range(2*N)]
Rmat = [0]*(2*N)

M[0][1] = 1

M[2*N-1][2*N-2] = -3*k(N)
M[2*N-1][2*N-1] = -4*mu(4)/((dr*N)**3)
Rmat[2*N-1] = p - 3*k(N)*alpha(N)*DT[N-1]



for i in range(1, N):
    r = dr * i
    M[ 2*(i-1) + 1][ 2*(i-1) : 2*(i-1) + 4] = [-r, -1/sq(r), r, 1/sq(r)]

for i in range(1,N):
    r = dr*i
    M[2*(i-1)+2][2*(i-1) : 2*(i-1) + 4] = [-3*k(i), 4*mu(i)/(r**3), 3*k(i+1), -4*mu(i+1)/(r**3)]

    Rmat[2*(i-1)+2] = -3*k(i)*alpha(i)*DT[i-1]+3*k(i+1)*alpha(i+1)*DT[i]

In [51]:
def Tcontraintesi(sol_ab,i,r) -> ndarray:
    '''Calcule de la contrainte a partir des solution
    Attention i>0'''

    sigmai=zeros((3,3))
    ai,bi=sol_ab[2*i-2],sol_ab[2*i-1]
    sigmai[0][0]=ai*r+bi
    sigmai[1][1]=1.5*ai*r+bi
    sigmai[2][2]=1.5*ai*r+bi

    return sigmai



def Tdeplacementsi(sol_ab,i,r) -> ndarray:
    '''Calcule de la contrainte a partir des solution
    Attention i>0'''

    epsilonai=zeros((3,3))
    ai,bi=sol_ab[2*i-2],sol_ab[2*i-1]
    epsilonai[0][0]=ai - 2 * bi / (r**3)
    epsilonai[1][1]=ai + bi/(r**3)
    epsilonai[2][2]=ai + bi/(r**3)

    return epsilonai

In [54]:
sol = linalg.solve(M, Rmat)

X = linspace(0, R, N)
Y1, Y2, Y3 = [], [], []

for i in range(1, N+1):
    r = i*dr
    contrainte = Tdeplacementsi(sol, i, r)
    Y1.append(contrainte[0][0])
    Y2.append(contrainte[1][1])
    Y3.append(contrainte[2][2])



px.line(x = X, y = [Y1, Y2, Y3])

In [53]:
a = sol[::2]
b = sol[1::2]
print(a)
print(b)

[-0.00061472 -0.00061477 -0.00061491 -0.00061514 -0.00061547 -0.00061589
 -0.00061641 -0.00061702 -0.00061772 -0.00061852 -0.00061941 -0.00062039
 -0.00062147 -0.00062264 -0.00062391 -0.00062527 -0.00062672 -0.00062827
 -0.00062991 -0.00063165 -0.00063347 -0.0006354  -0.00063741 -0.00063952
 -0.00064172 -0.00064402 -0.00064641 -0.0006489  -0.00065148 -0.00065415
 -0.00065691 -0.00065977 -0.00066273 -0.00066578 -0.00066892 -0.00067215
 -0.00067548 -0.0006789  -0.00068242 -0.00068603 -0.00068973 -0.00069353
 -0.00069742 -0.00070141 -0.00070548 -0.00070966 -0.00071392 -0.00071828
 -0.00072274 -0.00072729 -0.00073193 -0.00073666 -0.00074149 -0.00074641
 -0.00075143 -0.00075654 -0.00076174 -0.00076704 -0.00077243 -0.00077792
 -0.0007835  -0.00078917 -0.00079494 -0.0008008  -0.00080675 -0.0008128
 -0.00081894 -0.00082518 -0.0008315  -0.00083793 -0.00084444 -0.00085105
 -0.00085776 -0.00086456 -0.00087145 -0.00087843 -0.00088551 -0.00089269
 -0.00089995 -0.00090731 -0.00091477 -0.00092232 -0.